Importing Lib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import time

In [ ]:
data_dir1=r'/content/drive/MyDrive/data/RiceLeafs/train'
data_dir2=r'/content/drive/MyDrive/data/RiceLeafs/validation'

Pre-Processing

In [ ]:
import pathlib
batch_size = 64
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir1,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir2,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 2684 files belonging to 4 classes.
Using 2148 files for training.
Found 671 files belonging to 4 classes.
Using 134 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['BrownSpot', 'Healthy', 'Hispa', 'LeafBlast']


Tuning & Normalization

In [ ]:
UTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=UTOTUNE)
val_ds = train_ds = train_ds.cache().prefetch(buffer_size=UTOTUNE)

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./256)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x),y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Modeling

In [ ]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

In [ ]:
# def squeezenet(input_shape, n_classes):
#   # 1.0 version

#   def fire(m, fs, fe):
#     s = Conv2D(fs, 1, activation='relu')(m)
#     e1 = Conv2D(fe, 1, activation='relu')(s)
#     e3 = Conv2D(fe, 3, padding='same', activation='relu')(s)
#     output = Concatenate()([e1, e3])
#     return output

#   input = Input(input_shape)

#   m = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
#   m = MaxPool2D(3, strides=2, padding='same')(m)

#   m = fire(m, 16, 64)
#   m = fire(m, 16, 64)
#   m = fire(m, 32, 128)
#   m = MaxPool2D(3, strides=2, padding='same')(m)

#   m = fire(m, 32, 128)
#   m = fire(m, 48, 192)
#   m = fire(m, 48, 192)
#   m = fire(m, 64, 256)
#   m = MaxPool2D(3, strides=2, padding='same')(m)

#   m = fire(m, 64, 256)
#   m = Conv2D(n_classes, 1)(m)
#   m = GlobalAvgPool2D()(m)

#   output = Activation('softmax')(m)

#   model = Model(input, output)
#   return model

In [ ]:
def squeezenet(input_shape, n_classes):
  # 1.1 version

  def fire(m, fs, fe):
    s = Conv2D(fs, 1, activation='relu')(m)
    e1 = Conv2D(fe, 1, activation='relu')(s)
    e3 = Conv2D(fe, 3, padding='same', activation='relu')(s)
    output = Concatenate()([e1, e3])
    return output

  input = Input(input_shape)

  m = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
  m = MaxPool2D(3, strides=2, padding='same')(m)

  m = fire(m, 16, 64)
  m = fire(m, 16, 64)
  m = MaxPool2D(3, strides=2, padding='same')(m)

  m = fire(m, 32, 128)
  m = fire(m, 32, 128)
  m = MaxPool2D(3, strides=2, padding='same')(m)

  m = fire(m, 48, 192)
  m = fire(m, 48, 192)
  m = fire(m, 64, 256)
  m = fire(m, 64, 256)
  m = Conv2D(n_classes, 1)(m)
  m = GlobalAvgPool2D()(m)

  output = Activation('softmax')(m)

  model = Model(input, output)
  return model

In [ ]:
input_shape=(img_height, img_width, 3)
n_classes =2
K.clear_session()
model = squeezenet(input_shape, n_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 96  14208       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 64, 64, 96)   0           ['conv2d[0][0]']                 
                                                                                              

In [ ]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
opt = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.1)

model.compile(optimizer=opt,
              loss='mse',
              metrics=['accuracy'])

In [ ]:
# model.compile(optimizer='sgd',
#               loss='mse',
#               metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
34/34 [==============================] - 26s 250ms/step - loss: 2.2254 - accuracy: 0.4334 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch 2/10
34/34 [==============================] - 6s 180ms/step - loss: 2.2384 - accuracy: 0.4432 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch 3/10
34/34 [==============================] - 7s 210ms/step - loss: 2.2384 - accuracy: 0.4432 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch 4/10
34/34 [==============================] - 7s 207ms/step - loss: 2.2384 - accuracy: 0.4432 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch 5/10
34/34 [==============================] - 6s 179ms/step - loss: 2.2384 - accuracy: 0.4432 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch 6/10
34/34 [==============================] - 6s 181ms/step - loss: 2.2384 - accuracy: 0.4432 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch 7/10
34/34 [==============================] - 6s 179ms/step - loss: 2.2384 - accuracy: 0.4432 - val_loss: 2.2384 - val_accuracy: 0.4432
Epoch